In [3]:
import subprocess
import glob
import os
from os.path import exists
import re
#from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import Pool

In [4]:
! plotProfile

usage: plotProfile -m matrix.gz
help: plotProfile -h / plotProfile --help
plotProfile: error: the following arguments are required: --outFileName/-out/-o


In [5]:
output_dir = '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir'


In [11]:
bams_output = os.path.join(output_dir, '4_intersect_bam')
bam_files = glob.glob(os.path.join(bams_output, '*.bam'))
bam_files

['/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/C12C_clean_filtered.bam',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/C12A_clean_filtered.bam',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/DM2_clean_filtered.bam',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/DM1_clean_filtered.bam',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/DM3_clean_filtered.bam',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/C12B_clean_filtered.bam']

In [12]:
def bam_to_bigWig(bam_file, output_dir):
    bigwig_file = os.path.join(output_dir, os.path.basename(bam_file).replace('.bam', '.bw'))
    cmd = [
        'bamCoverage', 
        '-b', bam_file, 
        '-o', bigwig_file,
        '--normalizeUsing', 'RPKM',  # Normalize the coverage (you can use other methods as well)
        '--binSize', '10'  # The size of the bins, in bases
    ]
    subprocess.run(cmd, check=True)
    return bigwig_file

# Wrapper function to pass multiple arguments to the mapped function
def wrapper_bam_to_bigWig(args):
    return bam_to_bigWig(*args)

bw_output = os.path.join(output_dir, '6_bw')
if not os.path.exists(bw_output):
    os.makedirs(bw_output)


# Use a Pool to process BAM files in parallel
with Pool(4) as p:  # Here 4 represents the number of processes to run concurrently
    p.map(wrapper_bam_to_bigWig, [(bam_file, bw_output) for bam_file in bam_files])


normalization: RPKM
bamFilesList: ['/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/C12C_clean_filtered.bam']
binLength: 10
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkSize: None
defaultFragmentLength: read length
numberOfProcessors: 1
verbose: False
region: None
bedFile: None
minMappingQuality: None
ignoreDuplicates: False
chrsToSkip: []
stepSize: 10
center_read: False
samFlag_include: None
samFlag_exclude: None
minFragmentLength: 0
maxFragmentLength: 0
zerosToNans: False
smoothLength: None
save_data: False
out_file_for_raw_data: None
maxPairedFragmentLength: 1000
normalization: RPKM
bamFilesList: ['/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/DM2_clean_filtered.bam']
binLength: 10
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkSize: None
defaultFragmentLength: read length
numberO

In [13]:
peaks_output = os.path.join(output_dir, '5_macs2peaks_simple')
bed_files = glob.glob(os.path.join(peaks_output, '*.bed'))
bw_files = glob.glob(os.path.join(output_dir, '6_bw', '*.bw'))

In [14]:
bw_files

['/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/6_bw/C12B_clean_filtered.bw',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/6_bw/C12C_clean_filtered.bw',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/6_bw/DM2_clean_filtered.bw',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/6_bw/DM1_clean_filtered.bw',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/6_bw/C12A_clean_filtered.bw',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/6_bw/DM3_clean_filtered.bw']

In [15]:
def run_deepTools(bed_file, bigwig_file, output_dir):
    sorted_bed = os.path.join(output_dir, os.path.basename(bed_file).replace('.bed', '_sorted.bed'))
    matrix_file = os.path.join(output_dir, os.path.basename(bed_file).replace('.bed', '_matrix.gz'))
    heatmap_file = os.path.join(output_dir, os.path.basename(bed_file).replace('.bed', '_heatmap.pdf'))
    profile_file = os.path.join(output_dir, os.path.basename(bed_file).replace('.bed', '_profile.pdf'))

    # Sort the BED file
    cmd = ['sort', '-k1,1', '-k2,2n', bed_file, '-o', sorted_bed]
    subprocess.run(cmd, check=True)

    # Compute the matrix
    cmd = [
        'computeMatrix', 'reference-point',
        '--referencePoint', 'center',
        '-R', sorted_bed,
        '-S', bigwig_file,
        '-a', '1000',
        '-b', '1000',
        '-out', matrix_file
    ]
    subprocess.run(cmd, check=True)

    # Plot the heatmap
    cmd = [
        'plotHeatmap', '-m', matrix_file,
        '-out', heatmap_file,
        '--colorMap', 'RdBu_r',
        # '--zMin', '-20',
        # '--zMax', '20',
        '--sortUsing', 'mean',
        '--missingDataColor', '0.5',
        '--heatmapHeight', '10'
    ]
    subprocess.run(cmd, check=True)

    # Plot the profile
    cmd = [
        'plotProfile', '-m', matrix_file,
        '-out', profile_file,
        '--numPlotsPerRow', '1',
        '--plotHeight', '5',
        '--plotWidth', '10'
    ]
    subprocess.run(cmd, check=True)

def wrapper_run_deepTools(args):
    return run_deepTools(*args)


bedtools_output = os.path.join(output_dir, '7_sorted_bed_and_matrix_auto2')
if not os.path.exists(bedtools_output):
    os.makedirs(bedtools_output)


with Pool(4) as p:  # Using 4 concurrent processes
    p.map(wrapper_run_deepTools, [(bed_file, bw_file, bedtools_output) for bed_file, bw_file in zip(bed_files, bw_files)])

Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/hu/miniconda3/envs/work2/bin/computeMatrix", line 10, in <module>
  File "/home/hu/miniconda3/envs/work2/bin/computeMatrix", line 10, in <module>
Traceback (most recent call last):
  File "/home/hu/miniconda3/envs/work2/bin/computeMatrix", line 10, in <module>
    sys.exit(main())
    sys.exit(main())
  File "/home/hu/miniconda3/envs/work2/lib/python3.10/site-packages/deeptools/computeMatrix.py", line 428, in main
  File "/home/hu/miniconda3/envs/work2/lib/python3.10/site-packages/deeptools/computeMatrix.py", line 428, in main
Traceback (most recent call last):
  File "/home/hu/miniconda3/envs/work2/bin/computeMatrix", line 10, in <module>
    sys.exit(main())
  File "/home/hu/miniconda3/envs/work2/lib/python3.10/site-packages/deeptools/computeMatrix.py", line 428, in main
    sys.exit(main())
  File "/home/hu/miniconda3/envs/work2/lib/python3.10/site-packages/deeptools/computeMatrix.py", line 428, in m

KeyboardInterrupt: 

In [18]:
def run_deepTools(bed_file, bigwig_file, output_dir):
    sorted_bed = os.path.join(output_dir, os.path.basename(bed_file).replace('.bed', '_sorted.bed'))
    matrix_file = os.path.join(output_dir, os.path.basename(bed_file).replace('.bed', '_matrix.gz'))
    heatmap_file = os.path.join(output_dir, os.path.basename(bed_file).replace('.bed', '_heatmap.pdf'))
    profile_file = os.path.join(output_dir, os.path.basename(bed_file).replace('.bed', '_profile.pdf'))

    # Sort the BED file
    cmd = ['sort', '-k1,1', '-k2,2n', bed_file, '-o', sorted_bed]
    subprocess.run(cmd, check=True)

    # Compute the matrix
    cmd = [
        'computeMatrix', 'reference-point',
        '--referencePoint', 'center',
        '-R', sorted_bed,
        '-S', bigwig_file,
        '-a', '1000',
        '-b', '1000',
        '-out', matrix_file
    ]
    subprocess.run(cmd, check=True)

    # Plot the heatmap
    cmd = [
        'plotHeatmap', '-m', matrix_file,
        '-out', heatmap_file,
        '--colorMap', 'RdBu_r',
        '--zMin', '0',
        '--zMax', '380',
        '--sortUsing', 'mean',
        '--missingDataColor', '0.5',
        '--heatmapHeight', '8'
    ]
    subprocess.run(cmd, check=True)

    # Plot the profile
    cmd = [
        'plotProfile', '-m', matrix_file,
        '-out', profile_file,
        '--numPlotsPerRow', '1',
        '--plotHeight', '5',
        '--plotWidth', '10'
    ]
    subprocess.run(cmd, check=True)

def wrapper_run_deepTools(args):
    return run_deepTools(*args)


bedtools_output = os.path.join(output_dir, '7_sorted_bed_and_matrix_zMax380')
if not os.path.exists(bedtools_output):
    os.makedirs(bedtools_output)


with Pool(4) as p:  # Using 4 concurrent processes
    p.map(wrapper_run_deepTools, [(bed_file, bw_file, bedtools_output) for bed_file, bw_file in zip(bed_files, bw_files)])

In [17]:
import os
import subprocess
from multiprocessing import Pool

def run_deepTools(bed_file, bigwig_file, output_dir, ref_point='TSS'):
    sorted_bed = os.path.join(output_dir, os.path.basename(bed_file).replace('.bed', '_sorted.bed'))
    matrix_file = os.path.join(output_dir, os.path.basename(bed_file).replace('.bed', f'_{ref_point}_matrix.gz'))
    heatmap_file = os.path.join(output_dir, os.path.basename(bed_file).replace('.bed', f'_{ref_point}_heatmap.pdf'))
    profile_file = os.path.join(output_dir, os.path.basename(bed_file).replace('.bed', f'_{ref_point}_profile.pdf'))

    # Sort the BED file
    cmd = ['sort', '-k1,1', '-k2,2n', bed_file, '-o', sorted_bed]
    subprocess.run(cmd, check=True)

    # Compute the matrix
    ref_point_option = 'TSS' if ref_point == 'TSS' else 'TES'
    cmd = [
        'computeMatrix', 'reference-point',
        '--referencePoint', ref_point_option,
        '-R', sorted_bed,
        '-S', bigwig_file,
        '-a', '1000',
        '-b', '1000',
        '-out', matrix_file
    ]
    subprocess.run(cmd, check=True)

    # Plot the heatmap
    cmd = [
        'plotHeatmap', '-m', matrix_file,
        '-out', heatmap_file,
        '--colorMap', 'RdBu_r',
        '--zMin', '0',
        '--zMax', '380',
        '--sortUsing', 'mean',
        '--missingDataColor', '0.5',
        '--heatmapHeight', '8'
    ]
    subprocess.run(cmd, check=True)

    # Plot the profile
    cmd = [
        'plotProfile', '-m', matrix_file,
        '-out', profile_file,
        '--numPlotsPerRow', '1',
        '--plotHeight', '5',
        '--plotWidth', '10'
    ]
    subprocess.run(cmd, check=True)

def wrapper_run_deepTools(args):
    return run_deepTools(*args)

output_dir = '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir'
bedtools_output = os.path.join(output_dir, '7_sorted_bed_and_matrix_zMax380_TSS_TTS')
if not os.path.exists(bedtools_output):
    os.makedirs(bedtools_output)

# Example usage, specifying TSS or TTS
with Pool(4) as p:  # Using 4 concurrent processes
    tasks = [(bed_file, bw_file, bedtools_output, 'TSS') for bed_file, bw_file in zip(bed_files, bw_files)]  # for TSS
    tasks += [(bed_file, bw_file, bedtools_output, 'TTS') for bed_file, bw_file in zip(bed_files, bw_files)]  # for TTS
    p.map(wrapper_run_deepTools, tasks)


Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/hu/miniconda3/envs/work2/bin/computeMatrix", line 10, in <module>
Traceback (most recent call last):
  File "/home/hu/miniconda3/envs/work2/bin/computeMatrix", line 10, in <module>
  File "/home/hu/miniconda3/envs/work2/bin/computeMatrix", line 10, in <module>
Traceback (most recent call last):
  File "/home/hu/miniconda3/envs/work2/bin/computeMatrix", line 10, in <module>
    sys.exit(main())
  File "/home/hu/miniconda3/envs/work2/lib/python3.10/site-packages/deeptools/computeMatrix.py", line 428, in main
    sys.exit(main())
  File "/home/hu/miniconda3/envs/work2/lib/python3.10/site-packages/deeptools/computeMatrix.py", line 428, in main
    sys.exit(main())
  File "/home/hu/miniconda3/envs/work2/lib/python3.10/site-packages/deeptools/computeMatrix.py", line 428, in main
    sys.exit(main())
  File "/home/hu/miniconda3/envs/work2/lib/python3.10/site-packages/deeptools/computeMatrix.py", line 428, in m

KeyboardInterrupt: 

In [ ]:
# Get the list of all BAM files in the directory
bam_files = [os.path.join(bams_output, f) for f in os.listdir(bams_output) if f.endswith('.bam')]
bam_files

In [6]:
bam_files = glob.glob(os.path.join(bams_output, '*.bam'))
bam_files

['/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/C12C_clean_filtered.bam',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/C12A_clean_filtered.bam',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/DM2_clean_filtered.bam',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/DM1_clean_filtered.bam',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/DM3_clean_filtered.bam',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/C12B_clean_filtered.bam']

In [17]:
merged_bam_dir = os.path.join(output_dir, '9_merged_bam')
if not os.path.exists(merged_bam_dir):
    os.makedirs(merged_bam_dir)

In [18]:
# Separate BAM files into two categories based on filename prefix: C12 and DM
c12_files = [f for f in bam_files if os.path.basename(f).startswith("C12")]
dm_files = [f for f in bam_files if os.path.basename(f).startswith("DM")]
c12_files
dm_files

['/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/DM2_clean_filtered.bam',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/DM1_clean_filtered.bam',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/DM3_clean_filtered.bam']

In [19]:
# Function to merge BAM files using samtools
def merge_bam_files(files, output_file):
    cmd = ['samtools', 'merge',  output_file] + files
    subprocess.run(cmd, check=True)

# Merge C12 and DM files
merge_bam_files(c12_files, os.path.join(merged_bam_dir, 'C12_merged.bam'))
merge_bam_files(dm_files, os.path.join(merged_bam_dir, 'DM_merged.bam'))

[bam_merge] File '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/9_merged_bam/C12_merged.bam' exists. Please apply '-f' to overwrite. Abort.


CalledProcessError: Command '['samtools', 'merge', '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/9_merged_bam/C12_merged.bam', '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/C12C_clean_filtered.bam', '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/C12A_clean_filtered.bam', '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/C12B_clean_filtered.bam']' returned non-zero exit status 1.

In [12]:
merged_bams = glob.glob(os.path.join(merged_bam_dir, '*.bam'))

In [13]:

def bamtobed(bam_file, output_dir):
    """
    Converts a BAM file to BED format.
    """
    # Set the output file path
    bed_file = os.path.join(output_dir, os.path.basename(bam_file).replace('.bam', '.bed'))
    
    # Convert BAM to BED
    cmd = ['bedtools', 'bamtobed', '-i', bam_file]
    with open(bed_file, 'w') as outfile:
        subprocess.run(cmd, stdout=outfile, check=True)
    
    return bed_file


def bamtobed_wrapper(args):
    return bamtobed(*args)


bam2bed_output = os.path.join(output_dir, '10_bam2bed')
if not os.path.exists(bam2bed_output):
    os.makedirs(bam2bed_output)

with Pool(2) as p:  # Using 4 concurrent processes
    p.map(bamtobed_wrapper, [(bam_file, bam2bed_output) for bam_file in merged_bams])


In [16]:
merged_bams = glob.glob(os.path.join(merged_bam_dir, '*.bam'))
merged_bams

NameError: name 'merged_bam_dir' is not defined

In [23]:
def index_bam(bam_file):
    command = [
        'sambamba',
        'index',
        '-t', '20',  # Number of threads
        bam_file
    ]
    
    # Execute the command
    result = subprocess.run(command, capture_output=True, text=True)

    # If there's an error, print the error message
    if result.returncode != 0:
        print(f"Error indexing {sample}:\n{result.stderr}")
        return  # Exit the function or handle the error as needed

# Index BAM files for all samples in parallel
with Pool(2) as p:
    p.map(index_bam, merged_bams)


In [20]:
bam_files = glob.glob(os.path.join(merged_bam_dir, '*.bam'))
bam_files.extend( glob.glob(os.path.join('/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/9_merged_bam', '*.bam')))

In [21]:
bam_files

['/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/9_merged_bam/DM_merged.bam',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/9_merged_bam/C12_merged.bam',
 '/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/9_merged_bam/C14_merged.bam',
 '/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/9_merged_bam/OK_merged.bam']

In [47]:
output_dir

'/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir'

In [51]:

def generate_normalized_bigwig(bam_file):
    norm_bam_dir = os.path.join(output_dir, '11_normalized_bigwigs')
    os.makedirs(norm_bam_dir, exist_ok=True)  # Create the directory if it doesn't exist
    
    bigwig_file = os.path.join(norm_bam_dir, os.path.basename(bam_file).replace('.bam', '.bw'))
    cmd = [
        'bamCoverage',
        '--bam', bam_file,
        '-o', bigwig_file,
        '--binSize', '10',
        '--normalizeUsing',  'RPGC',
        '--effectiveGenomeSize', '115000000',
        '--scaleFactor', '10',
        '--numberOfProcessors', '15'
    ]
    subprocess.run(cmd, check=True)
    return f'Generated normalized bigWig for {bam_file} at {bigwig_file}'


# Initialize the pool and generate normalized bigWigs
with Pool(4) as pool:
    normalization_results = pool.map(generate_normalized_bigwig, bam_files)

for result in normalization_results:
    print(result)


normalization: 1x (effective genome size 115000000)
normalization: 1x (effective genome size 115000000)
normalization: 1x (effective genome size 115000000)
normalization: 1x (effective genome size 115000000)
bamFilesList: ['/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/9_merged_bam/C14_merged.bam']
binLength: 10
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkSize: None
defaultFragmentLength: read length
numberOfProcessors: 15
verbose: False
region: None
bedFile: None
minMappingQuality: None
ignoreDuplicates: False
chrsToSkip: []
stepSize: 10
center_read: False
samFlag_include: None
samFlag_exclude: None
minFragmentLength: 0
maxFragmentLength: 0
zerosToNans: False
smoothLength: None
save_data: False
out_file_for_raw_data: None
maxPairedFragmentLength: 1000
bamFilesList: ['/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/9_merged_bam/OK_merged.bam']
binLength: 10
numberOfSamples: N

Generated normalized bigWig for /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/9_merged_bam/DM_merged.bam at /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/11_normalized_bigwigs/DM_merged.bw
Generated normalized bigWig for /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/9_merged_bam/C12_merged.bam at /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/11_normalized_bigwigs/C12_merged.bw
Generated normalized bigWig for /media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/9_merged_bam/C14_merged.bam at /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/11_normalized_bigwigs/C14_merged.bw
Generated normalized bigWig for /media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/9_merged_bam/OK_merged.bam at /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/11_normalized

In [ ]:
#just use rpkm 

In [22]:
bam_files

['/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/9_merged_bam/DM_merged.bam',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/9_merged_bam/C12_merged.bam',
 '/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/9_merged_bam/C14_merged.bam',
 '/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/9_merged_bam/OK_merged.bam']

In [57]:
def bam_to_bigWig(bam_file, output_dir):
    bigwig_file = os.path.join(output_dir, os.path.basename(bam_file).replace('.bam', '.bw'))
    cmd = [
        'bamCoverage', 
        '-b', bam_file, 
        '-o', bigwig_file,
        '--normalizeUsing', 'BPM',  # similar to TPM in RNA-seq
        '--binSize', '10',  # The size of the bins, in bases
        '--numberOfProcessors', '10'
    ]
    subprocess.run(cmd, check=True)
    return bigwig_file

# Wrapper function to pass multiple arguments to the mapped function
def wrapper_bam_to_bigWig(args):
    return bam_to_bigWig(*args)

merged_bam2bed_output = os.path.join(output_dir, '11_merged_bam2bw_BPM')
if not os.path.exists(merged_bam2bed_output):
    os.makedirs(merged_bam2bed_output)

with Pool(4) as p:  # Using 4 concurrent processes
    p.map(wrapper_bam_to_bigWig, [(bam_file, merged_bam2bed_output) for bam_file in bam_files])



normalization: BPM
normalization: BPM
bamFilesList: ['/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/9_merged_bam/DM_merged.bam']
binLength: 10
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkSize: None
defaultFragmentLength: read length
numberOfProcessors: 10
verbose: False
region: None
bedFile: None
minMappingQuality: None
ignoreDuplicates: False
chrsToSkip: []
stepSize: 10
center_read: False
samFlag_include: None
samFlag_exclude: None
minFragmentLength: 0
maxFragmentLength: 0
zerosToNans: False
smoothLength: None
save_data: False
out_file_for_raw_data: None
maxPairedFragmentLength: 1000
bamFilesList: ['/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/9_merged_bam/OK_merged.bam']
binLength: 10
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkSize: None
defaultFragmentLength: read length
numberOfProcessors: 10
verbose: False


In [57]:
def bam_to_bigWig(bam_file, output_dir):
    bigwig_file = os.path.join(output_dir, os.path.basename(bam_file).replace('.bam', '.bw'))
    cmd = [
        'bamCoverage', 
        '-b', bam_file, 
        '-o', bigwig_file,
        '--normalizeUsing', 'BPM',  # similar to TPM in RNA-seq
        '--binSize', '10',  # The size of the bins, in bases
        '--numberOfProcessors', '10'
    ]
    subprocess.run(cmd, check=True)
    return bigwig_file

# Wrapper function to pass multiple arguments to the mapped function
def wrapper_bam_to_bigWig(args):
    return bam_to_bigWig(*args)

merged_bam2bed_output = os.path.join(output_dir, '11_merged_bam2bw_BPM')
if not os.path.exists(merged_bam2bed_output):
    os.makedirs(merged_bam2bed_output)

with Pool(4) as p:  # Using 4 concurrent processes
    p.map(wrapper_bam_to_bigWig, [(bam_file, merged_bam2bed_output) for bam_file in bam_files])



normalization: BPM
normalization: BPM
bamFilesList: ['/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/9_merged_bam/DM_merged.bam']
binLength: 10
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkSize: None
defaultFragmentLength: read length
numberOfProcessors: 10
verbose: False
region: None
bedFile: None
minMappingQuality: None
ignoreDuplicates: False
chrsToSkip: []
stepSize: 10
center_read: False
samFlag_include: None
samFlag_exclude: None
minFragmentLength: 0
maxFragmentLength: 0
zerosToNans: False
smoothLength: None
save_data: False
out_file_for_raw_data: None
maxPairedFragmentLength: 1000
bamFilesList: ['/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/9_merged_bam/OK_merged.bam']
binLength: 10
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkSize: None
defaultFragmentLength: read length
numberOfProcessors: 10
verbose: False


In [23]:
bam_files

['/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/9_merged_bam/DM_merged.bam',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/9_merged_bam/C12_merged.bam',
 '/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/9_merged_bam/C14_merged.bam',
 '/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/9_merged_bam/OK_merged.bam']

### larger bin 

In [26]:
def bam_to_bigWig(bam_file, output_dir):
    bigwig_file = os.path.join(output_dir, os.path.basename(bam_file).replace('.bam', '.bw'))
    cmd = [
        'bamCoverage', 
        '-b', bam_file, 
        '-o', bigwig_file,
        '--normalizeUsing', 'BPM',  # similar to TPM in RNA-seq
        '--binSize', '30',  # The size of the bins, in bases
        '--numberOfProcessors', '10'
    ]
    subprocess.run(cmd, check=True)
    return bigwig_file

# Wrapper function to pass multiple arguments to the mapped function
def wrapper_bam_to_bigWig(args):
    return bam_to_bigWig(*args)

merged_bam2bed_output = os.path.join(output_dir, '11_merged_bam2bw_BPM_bin30')
if not os.path.exists(merged_bam2bed_output):
    os.makedirs(merged_bam2bed_output)

with Pool(4) as p:  # Using 4 concurrent processes
    p.map(wrapper_bam_to_bigWig, [(bam_file, merged_bam2bed_output) for bam_file in bam_files])



normalization: BPM
bamFilesList: ['/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/9_merged_bam/C14_merged.bam']
binLength: 30
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkSize: None
defaultFragmentLength: read length
numberOfProcessors: 10
verbose: False
region: None
bedFile: None
minMappingQuality: None
ignoreDuplicates: False
chrsToSkip: []
stepSize: 30
center_read: False
samFlag_include: None
samFlag_exclude: None
minFragmentLength: 0
maxFragmentLength: 0
zerosToNans: False
smoothLength: None
save_data: False
out_file_for_raw_data: None
maxPairedFragmentLength: 1000
normalization: BPM
bamFilesList: ['/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/9_merged_bam/DM_merged.bam']
binLength: 30
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkSize: None
defaultFragmentLength: read length
numberOfProcessors: 10
verbose: False

## For RNA

In [23]:
import os
import subprocess
from multiprocessing import Pool


In [36]:
def merge_bam(group_files, output_dir, group_name):
    merged_bam = os.path.join(output_dir, f"{group_name}_merged.bam")
    cmd = ["samtools", "merge", merged_bam] + group_files
    subprocess.run(cmd, check=True)
    print(f"Merged {group_name}: {merged_bam}")
    return merged_bam



In [32]:
bam_dir = "/media/HDD3/3DM/DM1_DM2d/2_bam"
output_dir = "/media/HDD3/3DM/DM1_DM2d/"
merged_dir = os.path.join(output_dir, "3_merged_bams")
bigwig_dir = os.path.join(output_dir, "4_bigwigs")

os.makedirs(merged_dir, exist_ok=True)
os.makedirs(bigwig_dir, exist_ok=True)

groups = ["COL", "DM"]
grouped_bam_files = [[group, [os.path.join(bam_dir, f) for f in os.listdir(bam_dir) if f.endswith(".bam") and f.startswith(group)]] for group in groups]
grouped_bam_files

[['COL',
  ['/media/HDD3/3DM/DM1_DM2d/2_bam/COL7_ROO_sorted.bam',
   '/media/HDD3/3DM/DM1_DM2d/2_bam/COL3_ROO_sorted.bam',
   '/media/HDD3/3DM/DM1_DM2d/2_bam/COL4_SHO_sorted.bam',
   '/media/HDD3/3DM/DM1_DM2d/2_bam/COL8_SHO_sorted.bam']],
 ['DM',
  ['/media/HDD3/3DM/DM1_DM2d/2_bam/DM5_ROOT_sorted.bam',
   '/media/HDD3/3DM/DM1_DM2d/2_bam/DM1_ROOT_sorted.bam',
   '/media/HDD3/3DM/DM1_DM2d/2_bam/DM6_SHOO_sorted.bam',
   '/media/HDD3/3DM/DM1_DM2d/2_bam/DM13_ROO_sorted.bam',
   '/media/HDD3/3DM/DM1_DM2d/2_bam/DM2_SHOO_sorted.bam',
   '/media/HDD3/3DM/DM1_DM2d/2_bam/DM14_SHO_sorted.bam']]]

In [33]:
grouped_bam_files = [['COL',
  [
      # '/media/HDD3/3DM/DM1_DM2d/2_bam/COL7_ROO_sorted.bam',
   # '/media/HDD3/3DM/DM1_DM2d/2_bam/COL3_ROO_sorted.bam',
   '/media/HDD3/3DM/DM1_DM2d/2_bam/COL4_SHO_sorted.bam',
   '/media/HDD3/3DM/DM1_DM2d/2_bam/COL8_SHO_sorted.bam']],
 ['DM',
  [
      # '/media/HDD3/3DM/DM1_DM2d/2_bam/DM5_ROOT_sorted.bam',
   # '/media/HDD3/3DM/DM1_DM2d/2_bam/DM1_ROOT_sorted.bam',
   '/media/HDD3/3DM/DM1_DM2d/2_bam/DM6_SHOO_sorted.bam',
   # '/media/HDD3/3DM/DM1_DM2d/2_bam/DM13_ROO_sorted.bam',
   '/media/HDD3/3DM/DM1_DM2d/2_bam/DM2_SHOO_sorted.bam',
   '/media/HDD3/3DM/DM1_DM2d/2_bam/DM14_SHO_sorted.bam']]]

In [34]:
num_processes = 2  # Adjust based on your system

with Pool(num_processes) as pool:
    merged_bams = pool.starmap(merge_bam, [(files, merged_dir, group) for group, files in grouped_bam_files])


Merged COL: /media/HDD3/3DM/DM1_DM2d/3_merged_bams/COL_merged.bam
Merged DM: /media/HDD3/3DM/DM1_DM2d/3_merged_bams/DM_merged.bam


In [38]:
! samtools index /media/HDD3/3DM/DM1_DM2d/3_merged_bams/COL_merged.bam

In [39]:
! samtools index /media/HDD3/3DM/DM1_DM2d/3_merged_bams/DM_merged.bam

In [1]:
! bamCoverage -b /media/HDD3/3DM/DM1_DM2d/3_merged_bams/COL_merged.bam -o\
    /media/HDD3/3DM/DM1_DM2d/4_bigwigs/COL_12_Shoot.bw\
    --binSize 10\
    --normalizeUsing RPGC\
    --effectiveGenomeSize 115000000\
    --scaleFactor 10\
    --numberOfProcessors 20

normalization: 1x (effective genome size 115000000)
bamFilesList: ['/media/HDD3/3DM/DM1_DM2d/3_merged_bams/COL_merged.bam']
binLength: 10
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkSize: None
defaultFragmentLength: read length
numberOfProcessors: 20
verbose: False
region: None
bedFile: None
minMappingQuality: None
ignoreDuplicates: False
chrsToSkip: []
stepSize: 10
center_read: False
samFlag_include: None
samFlag_exclude: None
minFragmentLength: 0
maxFragmentLength: 0
zerosToNans: False
smoothLength: None
save_data: False
out_file_for_raw_data: None
maxPairedFragmentLength: 1000


In [2]:
! bamCoverage -b /media/HDD3/3DM/DM1_DM2d/3_merged_bams/DM_merged.bam -o\
    /media/HDD3/3DM/DM1_DM2d/4_bigwigs/DM1_DM2d_Shoot.bw\
    --binSize 10\
    --normalizeUsing RPGC\
    --effectiveGenomeSize 115000000\
    --scaleFactor 10\
    --numberOfProcessors 20

normalization: 1x (effective genome size 115000000)
bamFilesList: ['/media/HDD3/3DM/DM1_DM2d/3_merged_bams/DM_merged.bam']
binLength: 10
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkSize: None
defaultFragmentLength: read length
numberOfProcessors: 20
verbose: False
region: None
bedFile: None
minMappingQuality: None
ignoreDuplicates: False
chrsToSkip: []
stepSize: 10
center_read: False
samFlag_include: None
samFlag_exclude: None
minFragmentLength: 0
maxFragmentLength: 0
zerosToNans: False
smoothLength: None
save_data: False
out_file_for_raw_data: None
maxPairedFragmentLength: 1000


In [ ]:
bamCoverage -b /media/HDD3/3DM/DM1_DM2d/3_merged_bams/COL_merged.bam -o\
    /media/HDD3/3DM/DM1_DM2d/4_bigwigs/COL_12_Shoot.bw\
    --binSize 10\
    --normalizeUsing RPGC\
    --effectiveGenomeSize 115000000\
    --scaleFactor 10\
    --numberOfProcessors 20

In [5]:
import os
import subprocess
from multiprocessing import Pool

def convert_bam_to_bigwig(bam_file, output_dir, effective_genome_size, bin_size=10, scale_factor=10, num_processors=20):
    bw_file = os.path.join(output_dir, os.path.basename(bam_file).replace('.bam', '.bw'))
    cmd = [
        "bamCoverage", "-b", bam_file, "-o", bw_file,
        "--binSize", str(bin_size),
        "--normalizeUsing", "RPGC",
        "--effectiveGenomeSize", str(effective_genome_size),
        "--scaleFactor", str(scale_factor),
        "--numberOfProcessors", str(num_processors)
    ]
    subprocess.run(cmd, check=True)
    print(f"Converted {bam_file} to BigWig: {bw_file}")
    
bam_dir = "/media/HDD3/3DM/DM1_DM2d/2_bam"
bw_dir = "/media/HDD3/3DM/DM1_DM2d/4_bw_individuals"
effective_genome_size = 115000000  # Adjust as needed

os.makedirs(bw_dir, exist_ok=True)

bam_files = [os.path.join(bam_dir, f) for f in os.listdir(bam_dir) if f.endswith(".bam")]
bam_files


['/media/HDD3/3DM/DM1_DM2d/2_bam/COL7_ROO_sorted.bam',
 '/media/HDD3/3DM/DM1_DM2d/2_bam/DM5_ROOT_sorted.bam',
 '/media/HDD3/3DM/DM1_DM2d/2_bam/DM1_ROOT_sorted.bam',
 '/media/HDD3/3DM/DM1_DM2d/2_bam/COL3_ROO_sorted.bam',
 '/media/HDD3/3DM/DM1_DM2d/2_bam/DM6_SHOO_sorted.bam',
 '/media/HDD3/3DM/DM1_DM2d/2_bam/DM13_ROO_sorted.bam',
 '/media/HDD3/3DM/DM1_DM2d/2_bam/DM2_SHOO_sorted.bam',
 '/media/HDD3/3DM/DM1_DM2d/2_bam/COL4_SHO_sorted.bam',
 '/media/HDD3/3DM/DM1_DM2d/2_bam/DM14_SHO_sorted.bam',
 '/media/HDD3/3DM/DM1_DM2d/2_bam/COL8_SHO_sorted.bam']

In [8]:
num_processes = 5  # Adjust based on your system

with Pool(num_processes) as pool:
    pool.starmap(convert_bam_to_bigwig, [(bam_file, bw_dir, effective_genome_size) for bam_file in bam_files])


normalization: 1x (effective genome size 115000000)
normalization: 1x (effective genome size 115000000)
normalization: 1x (effective genome size 115000000)
normalization: 1x (effective genome size 115000000)
normalization: 1x (effective genome size 115000000)
bamFilesList: ['/media/HDD3/3DM/DM1_DM2d/2_bam/COL3_ROO_sorted.bam']
binLength: 10
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkSize: None
defaultFragmentLength: read length
numberOfProcessors: 20
verbose: False
region: None
bedFile: None
minMappingQuality: None
ignoreDuplicates: False
chrsToSkip: []
stepSize: 10
center_read: False
samFlag_include: None
samFlag_exclude: None
minFragmentLength: 0
maxFragmentLength: 0
zerosToNans: False
smoothLength: None
save_data: False
out_file_for_raw_data: None
maxPairedFragmentLength: 1000
bamFilesList: ['/media/HDD3/3DM/DM1_DM2d/2_bam/DM1_ROOT_sorted.bam']
binLength: 10
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False

Converted /media/HDD3/3DM/DM1_DM2d/2_bam/COL7_ROO_sorted.bam to BigWig: /media/HDD3/3DM/DM1_DM2d/4_bw_individuals/COL7_ROO_sorted.bw


normalization: 1x (effective genome size 115000000)


Converted /media/HDD3/3DM/DM1_DM2d/2_bam/COL3_ROO_sorted.bam to BigWig: /media/HDD3/3DM/DM1_DM2d/4_bw_individuals/COL3_ROO_sorted.bw


normalization: 1x (effective genome size 115000000)
bamFilesList: ['/media/HDD3/3DM/DM1_DM2d/2_bam/DM13_ROO_sorted.bam']
binLength: 10
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkSize: None
defaultFragmentLength: read length
numberOfProcessors: 20
verbose: False
region: None
bedFile: None
minMappingQuality: None
ignoreDuplicates: False
chrsToSkip: []
stepSize: 10
center_read: False
samFlag_include: None
samFlag_exclude: None
minFragmentLength: 0
maxFragmentLength: 0
zerosToNans: False
smoothLength: None
save_data: False
out_file_for_raw_data: None
maxPairedFragmentLength: 1000
bamFilesList: ['/media/HDD3/3DM/DM1_DM2d/2_bam/DM2_SHOO_sorted.bam']
binLength: 10
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkSize: None
defaultFragmentLength: read length
numberOfProcessors: 20
verbose: False
region: None
bedFile: None
minMappingQuality: None
ignoreDuplicates: False
chrsToSkip: []
ste

...

Converted /media/HDD3/3DM/DM1_DM2d/2_bam/DM1_ROOT_sorted.bam to BigWig: /media/HDD3/3DM/DM1_DM2d/4_bw_individuals/DM1_ROOT_sorted.bw


normalization: 1x (effective genome size 115000000)
bamFilesList: ['/media/HDD3/3DM/DM1_DM2d/2_bam/DM14_SHO_sorted.bam']
binLength: 10
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkSize: None
defaultFragmentLength: read length
numberOfProcessors: 20
verbose: False
region: None
bedFile: None
minMappingQuality: None
ignoreDuplicates: False
chrsToSkip: []
stepSize: 10
center_read: False
samFlag_include: None
samFlag_exclude: None
minFragmentLength: 0
maxFragmentLength: 0
zerosToNans: False
smoothLength: None
save_data: False
out_file_for_raw_data: None
maxPairedFragmentLength: 1000


Converted /media/HDD3/3DM/DM1_DM2d/2_bam/DM5_ROOT_sorted.bam to BigWig: /media/HDD3/3DM/DM1_DM2d/4_bw_individuals/DM5_ROOT_sorted.bw


normalization: 1x (effective genome size 115000000)
bamFilesList: ['/media/HDD3/3DM/DM1_DM2d/2_bam/COL8_SHO_sorted.bam']
binLength: 10
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkSize: None
defaultFragmentLength: read length
numberOfProcessors: 20
verbose: False
region: None
bedFile: None
minMappingQuality: None
ignoreDuplicates: False
chrsToSkip: []
stepSize: 10
center_read: False
samFlag_include: None
samFlag_exclude: None
minFragmentLength: 0
maxFragmentLength: 0
zerosToNans: False
smoothLength: None
save_data: False
out_file_for_raw_data: None
maxPairedFragmentLength: 1000


Converted /media/HDD3/3DM/DM1_DM2d/2_bam/DM2_SHOO_sorted.bam to BigWig: /media/HDD3/3DM/DM1_DM2d/4_bw_individuals/DM2_SHOO_sorted.bw
Converted /media/HDD3/3DM/DM1_DM2d/2_bam/DM13_ROO_sorted.bam to BigWig: /media/HDD3/3DM/DM1_DM2d/4_bw_individuals/DM13_ROO_sorted.bw
Converted /media/HDD3/3DM/DM1_DM2d/2_bam/DM14_SHO_sorted.bam to BigWig: /media/HDD3/3DM/DM1_DM2d/4_bw_individuals/DM14_SHO_sorted.bw
Converted /media/HDD3/3DM/DM1_DM2d/2_bam/COL4_SHO_sorted.bam to BigWig: /media/HDD3/3DM/DM1_DM2d/4_bw_individuals/COL4_SHO_sorted.bw
Converted /media/HDD3/3DM/DM1_DM2d/2_bam/COL8_SHO_sorted.bam to BigWig: /media/HDD3/3DM/DM1_DM2d/4_bw_individuals/COL8_SHO_sorted.bw
